In [90]:
import pymongo
import numpy as np
import pandas as pd
import h5py as h5
import scipy.sparse as ss
import pickle

import matplotlib as mpl
mpl.use('Agg')
import matplotlib.pyplot as plt


In [5]:
client = pymongo.MongoClient('localhost', 27017)
db = client['blockChains']
collection = 'neotransactions'

In [13]:
data = list(db[collection].find({}))

In [123]:
rowIndex = []
colIndex = []
freq = []
neo = []
neoGas = []

for d in data:
    if d['fromAddressInt'] != d['toAddressInt']:
        rowIndex.append(d['fromAddressInt'])
        colIndex.append(d['toAddressInt'])
        rowIndex.append(d['toAddressInt'])
        colIndex.append(d['fromAddressInt'])
        freq.append(1.0)
        freq.append(1.0)
        neo.append(d['transactionNeoValue'])
        neo.append(d['transactionNeoValue'])
        neoGas.append(d['transactionNeoGasValue'])
        neoGas.append(d['transactionNeoGasValue'])

In [124]:
print(rowIndex[0:10])
print(colIndex[0:10])
print(freq[0:10])
print(neo[0:10])
print(neoGas[0:10])
maxLen = max(max(rowIndex), max(colIndex)) + 1

[7, 8, 9, 10, 11, 12, 16, 17, 18, 19]
[8, 7, 10, 9, 12, 11, 17, 16, 19, 18]
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
[200, 200, 26, 26, 265, 265, 9966, 9966, 28, 28]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [125]:
freqMat = ss.coo_matrix((freq, (rowIndex, colIndex)), shape=[maxLen, maxLen]).tocsr().todense()
neoMat = ss.coo_matrix((neo, (rowIndex, colIndex)), shape=[maxLen, maxLen]).tocsr().todense()
neoGasMat = ss.coo_matrix((neoGas, (rowIndex, colIndex)), shape=[maxLen, maxLen]).tocsr().todense()

In [126]:
freqMat.shape

(1767, 1767)

In [133]:
freqMatDf = pd.DataFrame(freqMat)
#freqMatDf = freqMatDf[(freqMatDf.T != 0).any()]
#freqMatDf = freqMatDf.loc[:, (freqMatDf != 0).any(axis=0)]

neoMatDf = pd.DataFrame(neoMat)
#neoMatDf = neoMatDf[(neoMatDf.T != 0).any()]
#neoMatDf = neoMatDf.loc[:, (neoMatDf != 0).any(axis=0)]

neoGasMatDf = pd.DataFrame(neoGasMat)
#neoGasMatDf = neoGasMatDf[(neoGasMatDf.T != 0).any()]
#neoGasMatDf = neoGasMatDf.loc[:, (neoGasMatDf != 0).any(axis=0)]


In [134]:
fig = plt.figure(1, figsize=(5, 5), dpi=300, frameon=False)
#ax = fig.add_subplot(111, aspect='equal')
ax = plt.Axes(fig, (0,0,1,1))
ax.set_axis_off()
fig.add_axes(ax)
ax.grid(False)
ax.set_xticklabels("")
ax.set_yticklabels("")
ax.imshow(np.log10(freqMatDf.values / sum(freqMatDf.values)), cmap='hot', interpolation='nearest')
fig.savefig('freq.png',dpi=300)
fig.show()

fig = plt.figure(1, figsize=(5, 5), dpi=300, frameon=False)
#ax = fig.add_subplot(111, aspect='equal')
ax = plt.Axes(fig, (0,0,1,1))
ax.set_axis_off()
fig.add_axes(ax)
ax.grid(False)
ax.set_xticklabels("")
ax.set_yticklabels("")
ax.imshow(np.log10(neoMatDf.values / sum(neoMatDf.values)), cmap='hot', interpolation='nearest')
fig.savefig('neo.png',dpi=300)
fig.show()

fig = plt.figure(1, figsize=(5, 5), dpi=300, frameon=False)
#ax = fig.add_subplot(111, aspect='equal')
ax = plt.Axes(fig, (0,0,1,1))
ax.set_axis_off()
fig.add_axes(ax)
ax.grid(False)
ax.set_xticklabels("")
ax.set_yticklabels("")
ax.imshow(np.log10(neoGasMatDf.values / sum(neoGasMatDf.values)), cmap='hot', interpolation='nearest')
fig.savefig('neoGas.png',dpi=300)
fig.show()

/Users/k/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in true_divide
  if __name__ == '__main__':
/Users/k/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: RuntimeWarning: divide by zero encountered in log10
  if __name__ == '__main__':


/Users/k/anaconda3/lib/python3.6/site-packages/matplotlib/figure.py:459: UserWarning: matplotlib is currently using a non-GUI backend, so cannot show the figure
  "matplotlib is currently using a non-GUI backend, "
/Users/k/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in true_divide
/Users/k/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: RuntimeWarning: divide by zero encountered in log10


/Users/k/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:33: RuntimeWarning: invalid value encountered in true_divide
/Users/k/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:33: RuntimeWarning: divide by zero encountered in log10


In [137]:
with open('freq.pkl','wb') as f:
    pickle.dump(freqMatDf, f,True)

with open('neo.pkl','wb') as f:
    pickle.dump(neoMatDf, f,True)

with open('neoGas.pkl','wb') as f:
    pickle.dump(neoGasMat, f,True)
